In [62]:
import json
import os
import pandas as pd
import requests
from tqdm import tqdm
import psycopg2


In [63]:
conn = psycopg2.connect(dbname='pgstage', user='linguist', password='eDQGK0GCStlYlHNV', host='192.168.122.183')
cursor = conn.cursor()


In [20]:
word = []
#definition = []
local_word = []
examples = []
examples_local = []
parent = "./y_lookup/en_ru/"
for file in os.listdir(parent):
    with open(os.path.join(parent,file), "r") as f: 
        data = json.load(f)
        for definition in data['def']:
            word_current =  definition['text']
            for translation in definition['tr']: 
                word.append(word_current)
                local_word.append(translation['text'])
                ex_en = []
                ex_rus = []
                if 'ex' in translation:
                    for exmpl in translation['ex']:
                        #print(exmpl)
                        ex_en.append(exmpl['text'])
                        ex_rus.append(exmpl['tr'][0]['text'])
                examples.append(ex_en)
                examples_local.append(ex_rus)
                #print(translation)
                #print()
    

In [ ]:
def parse_from_yandex_lookup(directory):
    word = []
    local_word = []
    examples = []
    examples_local = []
    
    for file in os.listdir(directory):
        with open(os.path.join(directory,file), "r") as f: 
            data = json.load(f)
            for definition in data['def']:
                word_current =  definition['text']
                for translation in definition['tr']: 
                    word.append(word_current)
                    local_word.append(translation['text'])
                    ex_en = []
                    ex_rus = []
                    if 'ex' in translation:
                        for exmpl in translation['ex']:
                            #print(exmpl)
                            ex_en.append(exmpl['text'])
                            ex_rus.append(exmpl['tr'][0]['text'])
                    examples.append(ex_en)
                    examples_local.append(ex_rus)
    data = pd.DataFrame(list(zip(word,local_word,examples, examples_local)),columns =['word', 'local_word', 'examples','local_examples'])
    return  data


In [69]:
def get_word_forms(word):
    req = """SELECT DISTINCT
    content_words.word_id,content_words.word_lemma, content_words.word_value
    FROM content_words
    WHERE content_words.word_value !~ ('\W') AND
    (array_length(regexp_split_to_array(content_words.word_value, '[ ''-]'), 1) = 1) 
    and word_lemma != 0 and word_hash = calc_hash(' """ + word + """')"""
    cursor.execute(req)
    req_res = cursor.fetchone()
    if not req_res:
        print(word, " is not in db")
        return []
    else:
        #print(a[0])
        word_id = req_res[0]
    word_forms_request = """SELECT DISTINCT 
    content_words.word_value
    FROM content_words
    where word_lemma = """ + str(word_id) + """  and word_lemma != word_id """
    cursor.execute(word_forms_request)
    word_forms = []
    for word in cursor:
        #print(word[0])
        word_forms.append(word[0])
    return word_forms
get_word_forms("book")


['booker', 'booked', 'booking', 'books']

In [71]:
def parse_json(path, word, local_word,examples,examples_local):
    try:
        with open(path, "r") as f: 
            data = json.load(f)
            for definition in data['def']:
                word_current =  definition['text']
                for translation in definition['tr']: 
                    word.append(word_current)
                    local_word.append(translation['text'])
                    ex_en = []
                    ex_rus = []
                    if 'ex' in translation:
                        for exmpl in translation['ex']:
                            #print(exmpl)
                            ex_en.append(exmpl['text'])
                            ex_rus.append(exmpl['tr'][0]['text'])
                    examples.append(ex_en)
                    examples_local.append(ex_rus)
    except:
        pass

In [72]:
def parse_from_yandex_lookup_vs_wordforms(directory):
    word = []
    local_word = []
    examples = []
    examples_local = []
    words = pd.read_csv("test_words.csv", header = None)
    for word_i in words[0].items():
        print (word_i[1])
        file = word_i[1] + ".json"
        parse_json(os.path.join(directory,file),word, local_word,examples,examples_local)
        wordforms = get_word_forms(word_i[1].lower())
        for wf in wordforms:
            print(">>>", wf)
            file = wf + ".json"
            parse_json(os.path.join(directory,file),word, local_word,examples,examples_local)
        
    data = pd.DataFrame(list(zip(word,local_word,examples, examples_local)),columns =['word', 'local_word', 'examples','local_examples'])
    return  data
enru_wv = parse_from_yandex_lookup_vs_wordforms("./y_lookup/en_ru/")

word
>>> wording
>>> worded
>>> words
the
>>> thebed
>>> thest
>>> the bed
>>> thebest
>>> thes
and
>>> anding
>>> ands
>>> ood
>>> andren
>>> anded
>>> ander
that
>>> that led
>>> thats
>>> those
for
>>> forring
>>> fored
>>> fors
with
>>> withim
>>> withing
>>> withs
but
>>> buts
you
>>> youed
>>> yous
>>> youta
>>> youx
>>> youing
>>> youger
this
>>> thises
>>> this led
>>> these
not
>>> not bed
>>> notest
>>> notting
>>> nots
have
>>> have led
>>> haveta
>>> haveing
>>> had
>>> haved
>>> haves
>>> having
>>> havest
letter
>>> lettered
>>> letterd
>>> lettering
>>> letters
>>> lettera
plate
>>> plating
>>> plated
>>> plates
treat
>>> treats
>>> treated
>>> treaten
>>> treating
>>> treatest
dinner
>>> dinners
necessary
>>> necessarie
go
>>> gota
>>> gos
>>> gone
>>> went
>>> goed
>>> goes
>>> goren
>>> going
secret
>>> secrets
>>> secret
>>> secreter
table
>>> tables
>>> tabled
>>> tabling
>>> tabler
create
>>> creating
>>> creates
>>> createing
>>> created
warm
>>> warmest
>>> warme

>>> befores
begin
>>> begins
>>> began
>>> beginning
>>> beginer
>>> begining
>>> beginnest
>>> begun
>>> begin
behind
>>> behinds
below
>>> below
>>> belowed
>>> belows
best
>>> bester
>>> bests
>>> best best
>>> besting
>>> bestest
>>> bested
synchrophasotron
synchrophasotron  is not in db


In [75]:
enru_wv.to_csv("yandex_lookup_en_ru_with_wordforms.csv")

In [74]:
enru_wv.head(30)

,word,local_word,examples,local_examples
0,word,слово,"[last word, old english word, new word in art,...","[последнее слово, старое английское слово, нов..."
1,word,известие,[],[]
2,word,слова,[],[]
3,word,говорить,[],[]
4,word,сформулировать,[],[]
5,word,текстовый,[word processing document],[текстовый документ]
6,word,словесный,[word game],[словесная игра]
7,wording,формулировка,[exact wording],[точная формулировка]
8,wording,редакция,[new wording],[новая редакция]
9,wording,текст,[wording of the declaration],[текст декларации]


In [76]:
en_fr_wv = parse_from_yandex_lookup_vs_wordforms("./y_lookup/en_fr/")

word
>>> wording
>>> worded
>>> words
the
>>> thebed
>>> thest
>>> the bed
>>> thebest
>>> thes
and
>>> anding
>>> ands
>>> ood
>>> andren
>>> anded
>>> ander
that
>>> that led
>>> thats
>>> those
for
>>> forring
>>> fored
>>> fors
with
>>> withim
>>> withing
>>> withs
but
>>> buts
you
>>> youed
>>> yous
>>> youta
>>> youx
>>> youing
>>> youger
this
>>> thises
>>> this led
>>> these
not
>>> not bed
>>> notest
>>> notting
>>> nots
have
>>> have led
>>> haveta
>>> haveing
>>> had
>>> haved
>>> haves
>>> having
>>> havest
letter
>>> lettered
>>> letterd
>>> lettering
>>> letters
>>> lettera
plate
>>> plating
>>> plated
>>> plates
treat
>>> treats
>>> treated
>>> treaten
>>> treating
>>> treatest
dinner
>>> dinners
necessary
>>> necessarie
go
>>> gota
>>> gos
>>> gone
>>> went
>>> goed
>>> goes
>>> goren
>>> going
secret
>>> secrets
>>> secret
>>> secreter
table
>>> tables
>>> tabled
>>> tabling
>>> tabler
create
>>> creating
>>> creates
>>> createing
>>> created
warm
>>> warmest
>>> warme

>>> befores
begin
>>> begins
>>> began
>>> beginning
>>> beginer
>>> begining
>>> beginnest
>>> begun
>>> begin
behind
>>> behinds
below
>>> below
>>> belowed
>>> belows
best
>>> bester
>>> bests
>>> best best
>>> besting
>>> bestest
>>> bested
synchrophasotron
synchrophasotron  is not in db


In [78]:
en_fr_wv.to_csv("yandex_lookup_en_fr_with_wordforms.csv")

In [21]:
len(word), len(local_word), len(examples), len(examples_local)

(676, 676, 676, 676)

In [35]:
enru = parse_from_yandex_lookup("./y_lookup/en_ru/")
enru.head()

,word,local_word,examples,local_examples
0,for,для,"[for the bank, for last month, for one year, f...","[для банка, за последний месяц, на один год, н..."
1,for,ради,[for god],[ради бога]
2,for,в течение,[],[]
3,for,в отношении,[],[]
4,for,для целей,[],[]


In [ ]:
enru

In [38]:
enfr = parse_from_yandex_lookup("./y_lookup/en_fr/")
enfr.head()

,word,local_word,examples,local_examples
0,for,pour,[],[]
1,cafe,café,[],[]
2,follow,suivre,[],[]
3,follow,suivi,[],[]
4,follow,ensuit,[],[]


In [39]:
enru.to_csv("yandex_lookup_en_ru_lemmas_only.csv")

In [40]:
enfr.to_csv("yandex_lookup_en_fr_lemmas_only.csv")

In [44]:
url_y = "http://192.168.122.13:31436/GetTranslate"
h_y = {"apiVersion": "1.0.0", "text": "гайки и болты", "langPair": {"source": "ru", "target": "fr"}}
#h_y = {"apiVersion": "1.0.0", "text": "Landing to the mars", "langPair": "en-ru"}
r = requests.post(url = url_y, json=h_y)
data = r.json()
data

{'translate': 'écrous et boulons'}

In [50]:
def y_translate(text, from_lang, to_lang):
    url_y = "http://192.168.122.13:31436/GetTranslate"
    h_y = {"apiVersion": "1.0.0", "text": text, "langPair": {"source": from_lang, "target": to_lang}}
    r = requests.post(url = url_y, json=h_y)
    data = r.json()
    return data['translate']
y_translate("привет уеба", "ru", "fr")

'salut, connard.'

In [55]:
enru_small = parse_from_yandex_lookup("./y_lookup/small/")


In [79]:
def naive_translation(en_ru_dataframe, from_lang,to_lang):
    word = []
    local_word = []
    example = []
    local_example = []
    for element_ind in tqdm(range(len(en_ru_dataframe))):
        el_list = list(en_ru_dataframe.iloc[element_ind])
        #print(el_list)
        word.append(el_list[0])
        tr_local_word = y_translate(el_list[1],from_lang,to_lang)
        local_word.append(tr_local_word)
        ex = []
        loc_ex = []
        for example_i in el_list[2]:
            ex.append(example_i)
            ex_tr = y_translate(example_i,"en",to_lang)
            loc_ex.append(ex_tr)
        example.append(ex)
        local_example.append(loc_ex)
    
    data = pd.DataFrame(list(zip(word,local_word,example, local_example)),columns =['word', 'local_word', 'examples','local_examples'])
    return data

df = naive_translation(enru_wv,"ru","fr")
df.head()

100%|██████████| 1318/1318 [18:23<00:00,  1.17s/it]


,word,local_word,examples,local_examples
0,word,mot,"[last word, old english word, new word in art,...","[dernier mot, le vieux mot anglais, nouveau mo..."
1,word,nouvelle,[],[]
2,word,mot,[],[]
3,word,dire,[],[]
4,word,formuler,[],[]


In [80]:
df.to_csv("naive_transaltion_en_ru_fr_with_wordfroms.csv")